<a href="https://colab.research.google.com/github/xBrymer/FreeCodeCamp-Projects/blob/master/Machine-Learning/SMS-Spam-Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib as plt


In [ ]:
TRAIN_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/train-data.tsv"
TEST_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/valid-data.tsv"

train_file_path = tf.keras.utils.get_file("train-data.tsv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("valid-data.tsv", TEST_DATA_URL)

In [ ]:
train_df = pd.read_csv(train_file_path, sep='\t')
test_df = pd.read_csv(test_file_path, sep='\t')

column_labels = ['Label', 'Text']

train_df.columns, test_df.columns = column_labels, column_labels

In [ ]:
print(train_df.head())
print(test_df.head())

  Label                                               Text
0   ham                           you can never do nothing
1   ham  now u sound like manky scouse boy steve,like! ...
2   ham  mum say we wan to go then go... then she can s...
3   ham  never y lei... i v lazy... got wat? dat day ü ...
4   ham  in xam hall boy asked girl tell me the startin...
  Label                                               Text
0   ham         not much, just some textin'. how bout you?
1   ham  i probably won't eat at all today. i think i'm...
2   ham  don‘t give a flying monkeys wot they think and...
3   ham                                who are you seeing?
4   ham  your opinion about me? 1. over 2. jada 3. kusr...


In [ ]:
column_labels = ['Text', 'Label']

train_df = train_df.reindex(columns = column_labels)
test_df = test_df.reindex(columns = column_labels)

In [ ]:
def classify_label(label):
  if label == 'ham':
    return 0
  elif label == 'spam':
    return 1

In [ ]:
train_df['Target'] = list(map(classify_label, train_df['Label']))
test_df['Target'] = list(map(classify_label, test_df['Label'] ))

In [ ]:
train_df = train_df.drop('Label', axis = 1)
test_df = test_df.drop('Label', axis =1)

In [ ]:
print(train_df.head(10))
print()
print(test_df.head(10))


                                                Text  Target
0                           you can never do nothing       0
1  now u sound like manky scouse boy steve,like! ...       0
2  mum say we wan to go then go... then she can s...       0
3  never y lei... i v lazy... got wat? dat day ü ...       0
4  in xam hall boy asked girl tell me the startin...       0
5  genius what's up. how your brother. pls send h...       0
6              they finally came to fix the ceiling.       0
7  urgent! call 09066350750 from your landline. y...       1
8  now that you have started dont stop. just pray...       0
9  hello darlin ive finished college now so txt m...       0

                                                Text  Target
0         not much, just some textin'. how bout you?       0
1  i probably won't eat at all today. i think i'm...       0
2  don‘t give a flying monkeys wot they think and...       0
3                                who are you seeing?       0
4  your opinion about m

In [ ]:
x_train, y_train = np.asarray(train_df['Text']), np.asarray(train_df['Target'])

In [ ]:
x_test, y_test = np.asarray(test_df['Text']), np.asarray(test_df['Target'])

In [ ]:
import tensorflow_hub as hub

pretrained_model = 'http://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'
hub_layer = hub.KerasLayer(pretrained_model, input_shape = [], dtype= tf.string, trainable = True)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()

model.add(hub_layer)
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, validation_data= (x_test, y_test), epochs = 20)

Epoch 1/20
131/131 [==============================] - 1s 8ms/step - loss: 0.4060 - accuracy: 0.8217 - val_loss: 0.2640 - val_accuracy: 0.9058
Epoch 2/20
131/131 [==============================] - 1s 7ms/step - loss: 0.2128 - accuracy: 0.9337 - val_loss: 0.1804 - val_accuracy: 0.9267
Epoch 3/20
131/131 [==============================] - 1s 7ms/step - loss: 0.1484 - accuracy: 0.9579 - val_loss: 0.1418 - val_accuracy: 0.9454
Epoch 4/20
131/131 [==============================] - 1s 6ms/step - loss: 0.1144 - accuracy: 0.9696 - val_loss: 0.1184 - val_accuracy: 0.9576
Epoch 5/20
131/131 [==============================] - 1s 7ms/step - loss: 0.0904 - accuracy: 0.9765 - val_loss: 0.1008 - val_accuracy: 0.9626
Epoch 6/20
131/131 [==============================] - 1s 7ms/step - loss: 0.0723 - accuracy: 0.9799 - val_loss: 0.0893 - val_accuracy: 0.9698
Epoch 7/20
131/131 [==============================] - 1s 7ms/step - loss: 0.0583 - accuracy: 0.9847 - val_loss: 0.0799 - val_accuracy: 0.9705
Epoch 

In [ ]:
def predict_message(text):
  predict = float(model.predict([text]))

  if predict < 0.5:
    label = 'ham'
    return [predict, label]

  elif predict >= 0.5:
    label = 'spam'
    return [predict, label]


In [ ]:
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!


In [ ]:
cd '/content/drive/My Drive/External Datasets/Models'

/content/drive/My Drive/External Datasets/Models


In [ ]:
model.save('SMS-SPAM-CLASSIFICATION.h5')